# Setup

If you're running the notebook locally using the provided `uv` environment, Run the below command to install the notebook:

`uv add --dev ipykernel notebook`


In [1]:
# !pip install casadi

# What is CasADi ?

**CasADi: Computer Algebra System (CAS) and Algorithmic Differentiation (AD)** is a widely adopted open-source software framework for nonlinear optimization and optimal control. Key features:

- **Symbolic Framework**: It uses symbolic expressions to represent mathematical formulas, which allows for the automatic computation of exact derivatives using simple chain rule.

- **Algorithmic Differentiation (AD)**: This core functionality enables efficient gradient-based optimization by calculating derivatives automatically.

In [2]:
import casadi as ca

# 1. Symbolic "Hello World"

- In standard libraries like NumPy, variables hold values
- In CasADi, variables are symbolic placeholders. The framework builds a computational graph (an expression tree) of operations you perform on these symbols. This graph is what allows for **Algorithmic Differentiation (AD)**.

To start, we need to understand the two primary symbolic types:

Ref- https://web.casadi.org/docs/#symbolic-framework

- **`SX` (Sparse Symbolic)**: Built from scalar operations. Efficient for small-to-medium graphs. The graph is a sequence of scalar unary/binary operations.

- **`MX` (Matrix Symbolic)**: Built from matrix operations. More general, supports high-level linear algebra (like matrix multiplication or solves). The graph is a sequence of matrix operations.

<hr>

*For most Motion Planning tasks involving vehicle dynamics (which are ODEs), `SX` is often preferred for defining the dynamics because it generates very efficient C-code, while `MX` might be used for the optimization structure itself*
<hr>

Let's define a simple non-linear function $y = \sin(x^2)$ and ask CaSADi to compute its derivative:

$$\frac{dy}{dx} = 2x \cos(x^2)$$

In [3]:
# 1. Define a symbolic variable 'x'
x = ca.SX.sym('x')

# 2. Define a function y = f(x)
y = ca.sin(x**2)

# 3. Compute the Jacobian (derivative of y w.r.t x)
J = ca.jacobian(y, x)

print(f"y: {y}")
print(f"J: {J}")

y: sin(sq(x))
J: (cos(sq(x))*(x+x))


Notice that `(x+x)`? CaSADi is quite literal- it recorded the derivative of $x^2$ as $x+x$ (which is $2x$) based on the graph operations. It doesn't always simplify algebraically unless asked, but numerically, it's exact.

### The Solver Interface

To solve a problem, we use `ca.nlpsol`. It expects a dictionary defining the problem:

1. `x`: The decision variables (the unknowns we want to find)
2. `f`: The scalar cost function we want to minimize

and few other arguments:

3. *solver_name*: A name for the solver (e.g.: "solver")
4. *plugin*: The actual optimization solver. Throughout the course, we will use interior point optimization method `ipopt`

In [4]:
# Define the problem dictionary
# 'x': decision variables, 'f': the cost function
nlp_prob = {'x': x, 'f': y}

# Create the solver
solver = ca.nlpsol('solver', 'ipopt', nlp_prob)

Now that the solver is created, it's actually a function we can call. We just need to give it an initial guess (`x0`).

In [5]:
# Solve with an initial guess of 0
res = solver(x0=0)

print(f"Optimal solution: {res['x']}")


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.11, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        1

Total number of variables............................:        1
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality c

Debugging optimization problems often requires reading these logs to see if the solver is "stuck" or just finished successfully.

You can find the minimum value in two places:

1. **The Iteration History**: Look at the column labeled `objective`

    - At `iter 0` (initial guess), the objective was `1.0000000e+01`
    - At `iter 1` (solution), the objective dropped to `1.0000000e+00`

2. **The Final Summary**: Look at the block below `Number of Iterations....: 1`

    - There is a row specifically named `Objective...............:    1.0000000000000000e+00`
    
**Key columns to watch in the future:**

- `inf_pr` (Infeasibility Primal): How much your constraints are being violated. You want this to be close to 0

- `inf_du` (Infeasibility Dual): Roughly, how close the derivative is to zero (optimality). You also want this close to 0

# 2. Non-Linear Programming (NLP)

A Non-Linear Program (NLP) generally looks like this:

$$\begin{aligned}
\min_{w} \quad & J(w) && \text{(Cost / Objective)} \\
\text{s.t.} \quad & g(w) = 0 && \text{(Equality Constraints)} \\
& h(w) \leq 0 && \text{(Inequality Constraints)}
\end{aligned}$$

- In CasADi, we don't define "equality" or "inequality" directly in the symbolic graph.

- Instead, we define a generic constraint function vector, let's call it $g$, and then later set numerical bounds on it.

<hr>

## Simple Unconstrained Minimization

Find x and cost

$$
f(x) = x^2 + 6x -10
$$

In [6]:
x = ca.SX.sym('x')
f = x**2 - 6*x + 10

nlp = {"x": x, "f": f}
solver = ca.nlpsol("solver", "ipopt", nlp, {"ipopt.print_level": 0, "print_time": False})
sol = solver(x0=0)

print(f"Optimal Solution: x={sol['x']}")
print(f"Optimal Cost: f={sol['f']}")

Optimal Solution: x=3
Optimal Cost: f=1


### What if we want to find the roots ?

In [7]:
x = ca.SX.sym('x')
f = 0 # x**2 - 6*x + 10

nlp = {"x": x, "f": f, "g": (x**2 - 6*x + 10)}
solver = ca.nlpsol("solver", "ipopt", nlp, {"ipopt.print_level": 0, "print_time": False})
sol = solver(x0=0, lbg=0, ubg=0)

print(f"Optimal Solution: x={sol['x']}")
print(f"Optimal Cost: f={sol['f']}")

Optimal Solution: x=3
Optimal Cost: f=0


## Simple Constrained Minimization

\begin{aligned}
\min_{x \in \mathbb{R}} \quad & (x - 1)^2 \\
\text{s.t.} \quad & x^2 \le 1
\end{aligned}


In [8]:
x = ca.SX.sym("x")
f = (x - 1)**2
g = x**2

nlp = {"x":x, "f":f, "g":g}
opts = {
    "ipopt.print_level": 0,
    "print_time": False,
}

# opts = {
#     "ipopt.print_level": 0,
#     "print_time": False,
#     "ipopt.tol": 1e-10, # Default is ~1e-8
#     "ipopt.acceptable_tol": 1e-10
# }

solver = ca.nlpsol("solver", "ipopt", nlp, opts)
sol = solver(x0=0.9, lbg=-ca.inf, ubg=1)

print(f"Optimal Solution: x={sol['x']}")
print(f"Optimal Cost: f={sol['f']}")

Optimal Solution: x=0.999936
Optimal Cost: f=4.13444e-09


In [9]:
stats = solver.stats()

print(f"Solver Status:  {stats['return_status']}")
print(f"Iteration Count: {stats['iter_count']}")

Solver Status:  Solve_Succeeded
Iteration Count: 12


In [10]:
# solver.print_options()